# Librerías

In [36]:
#Hacer todos los imports
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs


# Primer DATASET: Peleadores


In [37]:
#Leyendo DataSet de Peleadores
estadisticas_peleador = pd.read_csv("../data/Fighter_stats/estadisticas_peleador.csv")
fuente_1 = "https://www.kaggle.com/datasets/maksbasher/ufc-complete-dataset-all-events-1996-2024/data"
estadisticas_peleador.head(11)

,name,wins,losses,height,weight,reach,stance,age,SLpM,sig_str_acc,SApM,str_def,td_avg,td_acc,td_def,sub_avg
0,Amanda Ribas,12.0,5.0,160.02,56.70,167.64,Orthodox,30.0,4.63,0.40,3.40,0.61,2.07,0.51,0.85,0.7
1,Rose Namajunas,13.0,6.0,165.10,56.70,165.10,Orthodox,31.0,3.69,0.41,3.51,0.63,1.38,0.47,0.59,0.5
2,Karl Williams,10.0,1.0,190.50,106.59,200.66,Orthodox,34.0,2.87,0.52,1.70,0.60,4.75,0.50,1.00,0.2
3,Justin Tafa,7.0,4.0,182.88,119.75,187.96,Southpaw,30.0,4.09,0.54,5.02,0.47,0.00,0.00,0.50,0.0
4,Edmen Shahbazyan,13.0,4.0,187.96,83.91,190.50,Orthodox,26.0,3.60,0.52,4.09,0.45,2.24,0.38,0.63,0.6
5,AJ Dobson,7.0,3.0,185.42,83.91,193.04,Orthodox,32.0,4.29,0.46,5.31,0.46,1.67,0.75,0.65,0.3
6,Payton Talbott,8.0,0.0,177.80,61.23,177.80,Switch,25.0,8.05,0.54,3.58,0.51,0.00,0.00,0.90,0.5
7,Cameron Saaiman,9.0,2.0,172.72,61.23,170.18,Southpaw,23.0,5.32,0.46,4.18,0.51,0.91,0.28,0.47,0.7
8,Billy Quarantillo,18.0,6.0,177.80,65.77,177.80,Orthodox,35.0,7.36,0.56,5.57,0.43,1.24,0.23,0.61,1.1
9,Youssef Zalal,14.0,5.0,177.80,65.77,182.88,Switch,27.0,2.88,0.49,1.73,0.65,2.28,0.34,0.60,1.3


In [3]:
#Comprobando los Missing-Values
estadisticas_peleador.info()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2479 entries, 0 to 2478
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         2478 non-null   object 
 1   wins         2478 non-null   float64
 2   losses       2478 non-null   float64
 3   height       2478 non-null   float64
 4   weight       2478 non-null   float64
 5   reach        1823 non-null   float64
 6   stance       2401 non-null   object 
 7   age          2318 non-null   float64
 8   SLpM         2478 non-null   float64
 9   sig_str_acc  2478 non-null   float64
 10  SApM         2478 non-null   float64
 11  str_def      2478 non-null   float64
 12  td_avg       2478 non-null   float64
 13  td_acc       2478 non-null   float64
 14  td_def       2478 non-null   float64
 15  sub_avg      2478 non-null   float64
dtypes: float64(14), object(2)
memory usage: 310.0+ KB


### Limpieza del primer DATASET apartir de una copia


In [4]:
#Creando la copia:
df_1 = estadisticas_peleador.copy()

In [5]:
#Diccionarios de traducción:

ingles_columnas1 = ['name', 'wins', 'losses', 'height', 'weight', 'reach', 'stance', 'age',
                    'SLpM', 'sig_str_acc', 'SApM', 'str_def', 'td_avg', 'td_acc', 'td_def',
                    'sub_avg']

español_columnas1= ["Nombre","Victorias","Derrotas","Altura","Peso","Alcance",
                    "Guardia","Edad","Golpes_min","Golpes_acer","Golpes_abs",
                    "Golpes_def","Derribos_med","Derribos_acer", "Derribos_def",
                    "Sumision_int"]



#df_1["Guardia"].unique()
guardia_dict = {'Orthodox':'Ortodoxo', 'Southpaw':'Zurdo', 'Switch':'Ambidiestro', 'Open Stance':'Guardia Baja'}


In [6]:
#Traduciendo:
df_1.rename(columns=dict(zip(ingles_columnas1, español_columnas1)), inplace=True)
df_1['Guardia'] = df_1['Guardia'].replace({**guardia_dict})

In [7]:
#Eliminando filas con algún missing-values en alguna columna:
df_1.dropna(subset=['Alcance', 'Guardia', 'Edad'], inplace=True)


In [8]:
#Comprobando si hay peleadores duplicados:
peleadores_duplicados = df_1['Nombre'].duplicated().sum()
df_1[df_1['Nombre'].duplicated(keep= False)]

,Nombre,Victorias,Derrotas,Altura,Peso,Alcance,Guardia,Edad,Golpes_min,Golpes_acer,Golpes_abs,Golpes_def,Derribos_med,Derribos_acer,Derribos_def,Sumision_int
382,Bruno Silva,23.0,10.0,182.88,83.91,187.96,Ortodoxo,34.0,4.43,0.51,5.33,0.43,1.09,0.27,0.73,0.0
638,Bruno Silva,13.0,5.0,162.56,56.70,165.10,Ortodoxo,34.0,3.05,0.46,3.09,0.59,2.57,0.30,0.64,0.3
1879,Edward Faaloloto,2.0,5.0,185.42,70.31,177.80,Ortodoxo,39.0,2.28,0.32,6.25,0.44,1.01,0.25,0.33,0.0
1897,Edward Faaloloto,2.0,5.0,177.80,70.31,177.80,Ortodoxo,39.0,2.28,0.32,6.25,0.44,1.01,0.25,0.33,0.0


Para Bruno silva me quedo con la fila 382 ya que tiene mas victorias, entendiendo que es un valor mas actualizado
y para Edward entiendo que simplemente hubo una confusión con su altura quedandome con la fila 1879.

In [9]:
#Eliminando peleadores duplicados:
df_1.drop([638, 1897], inplace=True)

Voy añadir una nueva columna con los paises de cada peleador a ser posible usando la téctica de Webscrapping de la wikipedia
con la libreria de Python BeautifulSoup

He dejado encapsulado en MainWebscrapping.py el ejecutable y los utils en webscrapping.py en la carpeta de Utils generando un ".csv" con los luchadores y su nacionalidad.

In [10]:
#Leyendo csv:
nacionalidad_peleador = pd.read_csv("./nacionalidades_luchadores.csv")
nacionalidad_peleador.head()

,Luchador,Nacionalidad
0,Amanda Ribas,Brasileña
1,Rose Namajunas,EstadounidenseLituana
2,Karl Williams,missingvalue
3,Justin Tafa,Neozelandés
4,Edmen Shahbazyan,Estadounidense


In [11]:
#Insertando columna "Pais" y guardando el csv:
nacionalidad = nacionalidad_peleador.iloc[:, 1]  
estadisticas_peleador = df_1.insert(1, 'Pais', nacionalidad)

In [12]:
estadisticas_peleador

He tenido muchos missing values, intentaré completarlos con otro webscrapping de Sherdog

In [13]:
# Eliminar filas con NaN solo en la columna 'Nacionalidad'
df_1_clean = df_1.dropna(subset=['Pais'])   
#Eliminando los MissingValues y guardando el CSV:
df_1_clean= df_1_clean[~df_1_clean['Pais'].str.contains('missingvalue', na=False)] 
df_1_clean

,Nombre,Pais,Victorias,Derrotas,Altura,Peso,Alcance,Guardia,Edad,Golpes_min,Golpes_acer,Golpes_abs,Golpes_def,Derribos_med,Derribos_acer,Derribos_def,Sumision_int
0,Amanda Ribas,Brasileña,12.0,5.0,160.02,56.70,167.64,Ortodoxo,30.0,4.63,0.40,3.40,0.61,2.07,0.51,0.85,0.7
1,Rose Namajunas,EstadounidenseLituana,13.0,6.0,165.10,56.70,165.10,Ortodoxo,31.0,3.69,0.41,3.51,0.63,1.38,0.47,0.59,0.5
3,Justin Tafa,Neozelandés,7.0,4.0,182.88,119.75,187.96,Zurdo,30.0,4.09,0.54,5.02,0.47,0.00,0.00,0.50,0.0
4,Edmen Shahbazyan,Estadounidense,13.0,4.0,187.96,83.91,190.50,Ortodoxo,26.0,3.60,0.52,4.09,0.45,2.24,0.38,0.63,0.6
6,Payton Talbott,Estadounidense,8.0,0.0,177.80,61.23,177.80,Ambidiestro,25.0,8.05,0.54,3.58,0.51,0.00,0.00,0.90,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1800,Zhang Tiequan,Peruana,15.0,4.0,172.72,70.31,175.26,Ortodoxo,45.0,1.23,0.36,2.14,0.51,1.95,0.58,0.75,3.4
1804,Stephan Bonnar,Estadounidense,15.0,9.0,193.04,92.99,200.66,Ortodoxo,46.0,2.76,0.38,3.01,0.52,1.32,0.40,0.60,1.0
1807,Aaron Simpson,Estadounidense Rusa,12.0,5.0,182.88,77.11,185.42,Ortodoxo,49.0,3.60,0.52,2.53,0.56,3.78,0.34,0.83,0.5
1808,Carlo Prater,Estadounidense,31.0,18.0,180.34,70.31,182.88,Ambidiestro,42.0,2.03,0.47,4.07,0.35,0.43,0.09,0.57,1.7


In [14]:
# Diccionario paises normalizados
pais_dict = {
    'estado': 'Estados Unidos', 'brasi': 'Brasil', 'mexic': 'México', 'finlan': 'Finlandia','pana': 'Panamá',
    'cuba': 'Cuba', 'esp': 'España', 'arme': 'Armenia', 'nige': 'Nigeria','rus': 'Rusia','uzb': 'Uzbekistán',
    'ingl': 'Inglaterra','chi': 'China', 'neoze': 'Nueva Zelanda', 'suda': 'Sudáfrica', 'marr': 'Marruecos',
    'austr': 'Australia','pola': 'Polonia','sue': 'Suecia','isra': 'Israel','gale': 'Gales','ecua': 'Ecuador',
    'brit': 'Inglaterra','esco': 'Escocia','ucra': 'Ucrania','suri': 'Surinam','fili': 'Filipinas','canad': 'Canadá',
    'afg': 'Afganistán','argen': 'Argentina','peru': 'Perú','tay': 'Tayikistán','georg': 'Georgia','jap': 'Japón',
    'fran': 'Francia','ita': 'Italia','ira': 'Irak','puerto': 'Puerto Rico','urug': 'Uruguay','tún': 'Túnez',
    'rum': 'Rumanía','let': 'Letonia','neerl': 'Holanda','irla': 'Irlanda','belg': 'Bélgica','costar': 'Costa Rica',
    'portu': 'Portugal','cro': 'Croacia','grie': 'Grecia','dane': 'Dinamarca','boliv': 'Bolivia','liban': 'Líbano',
    'kaz': 'Kazajistán','surcor': 'Corea del Sur','cameru': 'Camerún','bosn': 'Bosnia','guya': 'Guyana',
    'búlg': 'Bulgaria','colom': 'Colombia','isla': 'Islandia','tail': 'Tailandia', 'litu': 'Lituania', 
    'ale': 'Alemania','suiz': 'Suiza','serb': 'Serbia','congo': 'Congo','ghan': 'Ghana','molda': 'Moldavia',
    'kirg': 'Kirguistán','jama': 'Jamaica','holand': 'Holanda'}

In [15]:
#Remplazando Países normalizados:

def reemplazar_pais(pais):
    pais_lower = pais.lower() 
    for clave, nombre in pais_dict.items():
        if clave in pais_lower:
            return nombre
    return pais  

df_1_clean['Pais'] = df_1_clean['Pais'].apply(reemplazar_pais)

print(f"Paises únicos: {df_1_clean['Pais'].unique()}")





Paises únicos: ['Brasil' 'Estados Unidos' 'Nueva Zelanda' 'Sudáfrica' 'Marruecos'
 'México' 'Australia' 'Polonia' 'Nigeria' 'Suecia' 'Israel' 'Gales'
 'Ecuador' 'Inglaterra' 'Rusia' 'China' 'Cuba' 'Escocia' 'Ucrania'
 'Surinam' 'Filipinas' 'Canadá' 'Argentina' 'Perú' 'Tayikistán' 'España'
 'Georgia' 'Japón' 'Armenia' 'Tailandia' 'Francia' 'Lituania' 'Kazajistán'
 'Alemania' 'Suiza' 'Serbia' 'Congo' 'Ghana' 'Portugal' 'Uruguay'
 'Moldavia' 'Panamá' 'Corea del Sur' 'Camerún' 'Bosnia' 'Guyana'
 'Uzbekistán' 'Bulgaria' 'Colombia' 'Italia' 'Irak' 'Puerto Rico'
 'Islandia' 'Finlandia' 'Túnez' 'Rumanía' 'Kirguistán' 'Jamaica' 'Grecia'
 'Irlanda' 'Holanda' 'Afganistán' 'Bélgica' 'Costa Rica' 'Croacia'
 'Dinamarca' 'Bolivia']


In [16]:
#Añadiento Países normalizados al DataFrame
df_1_clean['Pais'] = df_1_clean['Pais'].apply(reemplazar_pais)

In [17]:
#Reseteamos el indice:
df_1_clean.reset_index(drop=True, inplace=True)
df_1_clean

,Nombre,Pais,Victorias,Derrotas,Altura,Peso,Alcance,Guardia,Edad,Golpes_min,Golpes_acer,Golpes_abs,Golpes_def,Derribos_med,Derribos_acer,Derribos_def,Sumision_int
0,Amanda Ribas,Brasil,12.0,5.0,160.02,56.70,167.64,Ortodoxo,30.0,4.63,0.40,3.40,0.61,2.07,0.51,0.85,0.7
1,Rose Namajunas,Estados Unidos,13.0,6.0,165.10,56.70,165.10,Ortodoxo,31.0,3.69,0.41,3.51,0.63,1.38,0.47,0.59,0.5
2,Justin Tafa,Nueva Zelanda,7.0,4.0,182.88,119.75,187.96,Zurdo,30.0,4.09,0.54,5.02,0.47,0.00,0.00,0.50,0.0
3,Edmen Shahbazyan,Estados Unidos,13.0,4.0,187.96,83.91,190.50,Ortodoxo,26.0,3.60,0.52,4.09,0.45,2.24,0.38,0.63,0.6
4,Payton Talbott,Estados Unidos,8.0,0.0,177.80,61.23,177.80,Ambidiestro,25.0,8.05,0.54,3.58,0.51,0.00,0.00,0.90,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
689,Zhang Tiequan,Perú,15.0,4.0,172.72,70.31,175.26,Ortodoxo,45.0,1.23,0.36,2.14,0.51,1.95,0.58,0.75,3.4
690,Stephan Bonnar,Estados Unidos,15.0,9.0,193.04,92.99,200.66,Ortodoxo,46.0,2.76,0.38,3.01,0.52,1.32,0.40,0.60,1.0
691,Aaron Simpson,Estados Unidos,12.0,5.0,182.88,77.11,185.42,Ortodoxo,49.0,3.60,0.52,2.53,0.56,3.78,0.34,0.83,0.5
692,Carlo Prater,Estados Unidos,31.0,18.0,180.34,70.31,182.88,Ambidiestro,42.0,2.03,0.47,4.07,0.35,0.43,0.09,0.57,1.7


In [18]:
#Guardando en csv:
#df_1_clean.to_csv("estadisticas_peleador_limpio.csv")

# Segundo DATASET: Enfrentamientos

In [19]:
#Leyendo DataSet de Enfrentamientos
enfrentamientos = pd.read_csv("../data/Large_set/Enfrentamientos.csv")
fuente_2 = "https://www.kaggle.com/datasets/maksbasher/ufc-complete-dataset-all-events-1996-2024/data"
enfrentamientos.head(11)

,event,date,location,r_fighter,b_fighter,status,r_kd,b_kd,r_str,b_str,r_td,b_td,r_sub,b_sub,weight_class,method,method_detailed,round,time
0,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Rose Namajunas,Amanda Ribas,win,0.0,0.0,93.0,83.0,1.0,4.0,0.0,0.0,Women's Flyweight,U-DEC,NaN,5.0,5:00
1,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Karl Williams,Justin Tafa,win,0.0,0.0,40.0,21.0,7.0,0.0,1.0,0.0,Heavyweight,U-DEC,NaN,3.0,5:00
2,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Edmen Shahbazyan,AJ Dobson,win,1.0,0.0,27.0,15.0,1.0,0.0,0.0,0.0,Middleweight,KO/TKO,Punches,1.0,4:33
3,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Payton Talbott,Cameron Saaiman,win,1.0,0.0,79.0,31.0,0.0,0.0,0.0,0.0,Bantamweight,KO/TKO,Punches,2.0,0:21
4,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Youssef Zalal,Billy Quarantillo,win,0.0,0.0,33.0,10.0,2.0,0.0,2.0,0.0,Featherweight,SUB,Rear Naked Choke,2.0,1:50
5,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Fernando Padilla,Luis Pajuelo,win,1.0,0.0,34.0,5.0,0.0,0.0,1.0,0.0,Featherweight,SUB,D'Arce Choke,1.0,2:45
6,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Trey Ogden,Kurt Holobaugh,win,0.0,0.0,35.0,29.0,3.0,0.0,0.0,0.0,Lightweight,U-DEC,NaN,3.0,5:00
7,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Julian Erosa,Ricardo Ramos,win,0.0,0.0,4.0,7.0,0.0,2.0,1.0,0.0,Featherweight,SUB,Guillotine Choke,1.0,2:15
8,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Miles Johns,Cody Gibson,win,0.0,0.0,25.0,18.0,2.0,1.0,0.0,0.0,Bantamweight,U-DEC,NaN,3.0,5:00
9,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Jarno Errens,Steven Nguyen,win,1.0,0.0,105.0,63.0,0.0,1.0,0.0,0.0,Featherweight,U-DEC,NaN,3.0,5:00


In [20]:
#Verificando los missing_values:
enfrentamientos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7582 entries, 0 to 7581
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   event            7497 non-null   object 
 1   date             7497 non-null   object 
 2   location         7497 non-null   object 
 3   r_fighter        7497 non-null   object 
 4   b_fighter        7497 non-null   object 
 5   status           7582 non-null   object 
 6   r_kd             7476 non-null   float64
 7   b_kd             7476 non-null   float64
 8   r_str            7476 non-null   float64
 9   b_str            7476 non-null   float64
 10  r_td             7476 non-null   float64
 11  b_td             7476 non-null   float64
 12  r_sub            7476 non-null   float64
 13  b_sub            7476 non-null   float64
 14  weight_class     7497 non-null   object 
 15  method           7497 non-null   object 
 16  method_detailed  3836 non-null   object 
 17  round         

In [21]:
#Trabajar en la copia:
df_2 = enfrentamientos.copy()

In [22]:
#Eliminando cualquier fila que tenga un solo missingvalue y las columns que no me interesan:
df_2.dropna(inplace=True)
df_2.drop(['status','r_kd','b_kd', 'r_str', 'b_str', 'r_td', 'b_td', 'r_sub', 'b_sub'], axis=1, inplace=True)

In [23]:
df_2

,event,date,location,r_fighter,b_fighter,weight_class,method,method_detailed,round,time
2,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Edmen Shahbazyan,AJ Dobson,Middleweight,KO/TKO,Punches,1.0,4:33
3,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Payton Talbott,Cameron Saaiman,Bantamweight,KO/TKO,Punches,2.0,0:21
4,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Youssef Zalal,Billy Quarantillo,Featherweight,SUB,Rear Naked Choke,2.0,1:50
5,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Fernando Padilla,Luis Pajuelo,Featherweight,SUB,D'Arce Choke,1.0,2:45
7,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Julian Erosa,Ricardo Ramos,Featherweight,SUB,Guillotine Choke,1.0,2:15
...,...,...,...,...,...,...,...,...,...,...
7491,UFC 2: No Way Out,3/11/1994,"Denver, Colorado, USA",Remco Pardoel,Alberta Cerra Leon,Open Weight,SUB,Ezekiel Choke,1.0,9:51
7493,UFC 2: No Way Out,3/11/1994,"Denver, Colorado, USA",Frank Hamaker,Thaddeus Luster,Open Weight,SUB,Keylock,1.0,4:52
7494,UFC 2: No Way Out,3/11/1994,"Denver, Colorado, USA",Johnny Rhodes,David Levicki,Open Weight,KO/TKO,Punches,1.0,12:13
7495,UFC 2: No Way Out,3/11/1994,"Denver, Colorado, USA",Patrick Smith,Ray Wizard,Open Weight,SUB,Guillotine Choke,1.0,0:58


Voy a cambiar todas las filas y columnas que estén en ingles para mayor comprensión a la hora de trabajarlas

In [24]:
#Diccionarios 

#Diccionario de columnas
ingles_columnas2 = ["event","date","location","r_fighter",
                    "b_fighter","weight_class","method",
                    "method_detailed","round","time"]

español_columnas2= ["Evento","Día","Donde",
                    "Ganador","Perdedor","Division",
                    "Metodo","Detalles","Round","Tiempo"]

#Todas las divisiones
division_dict = {'Middleweight': 'Peso Medio',
                'Bantamweight': 'Peso Gallo',
                'Featherweight': 'Peso Pluma',
                'Heavyweight': 'Peso Pesado',
                "Women's Bantamweight": 'Peso Gallo Femenino',
                'Lightweight': 'Peso Ligero',
                'Flyweight': 'Peso Mosca',
                "Women's Strawweight": 'Peso Paja Femenino',
                'Welterweight': 'Peso Welter',
                'Light Heavyweight': 'Peso Semipesado',
                "Women's Flyweight": 'Peso Mosca Femenino',
                'Catch Weight': 'Peso Pactado',
                "Women's Featherweight": 'Peso Pluma Femenino',
                'Super Heavyweight': 'Peso Super Pesado',
                'Open Weight': 'Peso Abierto'}

#Técnicas de Golpeo:
dict_golpes = {"Punches":"Golpes",
                "Kick":"Patada",
                "Punch":"Puño",
                "Elbows":"Codos",
                "Elbow":"Codo",
                "Flying Knee":"Rodilla Voladora",
                "Knees":"Rodillas",
                "Knee":"Rodilla",
                "Kicks":"Patadas",
                "Spinning Back Kick":"Patada Circular",
                "Spinning Back Fist":"Puñetazo Circular",
                "Spinning Back Elbow":"Codazo Circular",
                "Headbutts":"Cabezazos",
                "Slam":"Golpe Bajo"}

#Técnicas de Grappling y Estrangulaciones
dict_suelo = {"Rear Naked Choke":"Estrangulación Trasera",
                "D'Arce Choke":"Estrangulación D'Arce",
                "Guillotine Choke":"Estrangulación Guillotina",
                "Arm Triangle":"Triángulo de Brazo",
                "Armbar":"Llave de Brazo",
                "Triangle Choke":"Estrangulación Triangular",
                "Anaconda Choke":"Estrangulación Anaconda",
                "Neck Crank":"Torcedura de Cuello",
                "Kimura":"Llave Kimura",
                "Kneebar":"Llave de Rodilla",
                "Inverted Triangle" : "Triángulo Invertido",
                "Von Flue Choke" : "Estrangulación Von Flue",
                "Scarf Hold" : "Sujeción de Bufanda",
                "Straight Armbar" : "Llave de Brazo Recto",
                "Heel Hook" : "Llave de Talón",
                "Ankle Lock" : "Llave de Tobillo",
                "Forearm Choke" : "Estrangulación con Antebrazo",
                "Ezekiel Choke" : "Estrangulación Ezekiel",
                "Peruvian Necktie" : "Corbata Peruana",
                "Schultz Front Headlock" : "Llave de Cabeza Schultz",
                "Bulldog Choke" : "Estrangulación Bulldog",
                "Suloev Stretch" : "Estiramiento Suloev",
                "Omoplata" : "Omoplato",
                "Calf Slicer" : "Cortador de Pantorrilla",
                "North-South Choke" : "Estrangulación Norte-Sur",
                "Pace/Pillory Choke" : "Estrangulación Pace/Pillory",
                "Toe Hold" : "Llave Dedo del Pie",
                "Shoulder Choke" : "Estrangulación de Hombro",
                "Twister":"Torcedura"}

traduccion_dict = {**dict_golpes,**dict_suelo}


In [25]:
#Implementando los diccionarios:
df_2.rename(columns=dict(zip(ingles_columnas2, español_columnas2)), inplace=True)
df_2['Detalles'] = df_2['Detalles'].replace(traduccion_dict)
df_2['Division'] = df_2['Division'].replace({**division_dict})

In [26]:
#Voy a elimnar estas categorias
eliminar = ["Other","Other - Lock", "Other - Choke" ,"injury", "Gi Choke", "Keylock"]
eliminados = '|'.join(eliminar)
df_2 = df_2[~df_2['Detalles'].str.contains(eliminados, regex=True)]


In [27]:
df_2.head()

,Evento,Día,Donde,Ganador,Perdedor,Division,Metodo,Detalles,Round,Tiempo
2,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Edmen Shahbazyan,AJ Dobson,Peso Medio,KO/TKO,Golpes,1.0,4:33
3,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Payton Talbott,Cameron Saaiman,Peso Gallo,KO/TKO,Golpes,2.0,0:21
4,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Youssef Zalal,Billy Quarantillo,Peso Pluma,SUB,Estrangulación Trasera,2.0,1:50
5,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Fernando Padilla,Luis Pajuelo,Peso Pluma,SUB,Estrangulación D'Arce,1.0,2:45
7,UFC Fight Night: Ribas vs. Namajunas,3/23/2024,"Las Vegas, Nevada, USA",Julian Erosa,Ricardo Ramos,Peso Pluma,SUB,Estrangulación Guillotina,1.0,2:15


In [28]:
df_2.reset_index(drop=True, inplace=True)
#df_2.to_csv("Enfrentamientos_limpio.csv")


Crearé un DataFrame de Divisiones y sus respectivos rangos de peso

In [29]:
#Creando el dataframe:

informacion = {'Division':['Peso Medio','Peso Gallo','Peso Pluma','Peso Pesado','Peso Gallo Femenino','Peso Ligero',
                           'Peso Mosca','Peso Paja Femenino','Peso Welter','Peso Semipesado','Peso Mosca Femenino',
                           'Peso Pactado','Peso Pluma Femenino','Peso Super Pesado','Peso Abierto'],
               'Peso (kg)':['Hasta 84 kg','Hasta 61.2 kg','Hasta 65.8 kg','Hasta 120.2 kg','Hasta 61.2 kg','Hasta 70.3 kg',
                            'Hasta 56.7 kg','Hasta 52.2 kg','Hasta 77.1 kg','Hasta 93 kg','Hasta 56.7 kg','Peso pactado',
                            'Hasta 65.8 kg','Más de 120.2 kg','Cualquier peso']}
division = pd.DataFrame(informacion)

#division.to_csv("Categorias_por_peso.csv")

# Algunas estadísticas rapidas

In [30]:
#Donde se hacen mas eventos
df_2["Donde"].value_counts()[:3]

Donde
Las Vegas, Nevada, USA                        1299
Abu Dhabi, Abu Dhabi, United Arab Emirates      83
London, England, United Kingdom                 78
Name: count, dtype: int64

In [31]:
#Quien ha participado mas veces con las estadisticas recortadas obvio
df_2["Ganador"].value_counts()[:3]

Ganador
Charles Oliveira    20
Jim Miller          17
Donald Cerrone      16
Name: count, dtype: int64

In [32]:
#De que division ha habido mas combates con las estadisticas recortadas obvio
df_2["Division"].value_counts()[:3]

Division
Peso Ligero    640
Peso Welter    626
Peso Medio     569
Name: count, dtype: int64

In [33]:
#Mas KO's o sumisiones:
df_2 = df_2[~df_2['Metodo'].str.contains("DQ", regex=True)]
df_2["Metodo"].value_counts()[:3]

Metodo
KO/TKO    2330
SUB       1473
Name: count, dtype: int64

In [34]:
#Metodo mas efectivo
df_2["Detalles"].value_counts()[:3]

Detalles
Golpes                    931
Puño                      832
Estrangulación Trasera    576
Name: count, dtype: int64

In [35]:
#Round y tiempo mas veces visto...
frecuencias = df_2.groupby(['Round', 'Tiempo']).size().reset_index(name='count')
comun = frecuencias.loc[frecuencias['count'].idxmax()]